In [1]:
from pyrouge import Rouge155
from pprint import pprint

ref_texts = {'A': "Poor nations pressurise developed countries into granting trade subsidies.",
             'B': "Developed countries should be pressurized. Business exemptions to poor nations.",
             'C': "World's poor decide to urge developed nations for business concessions."}
summary_text = "Poor nations demand trade subsidies from developed nations."

In [6]:
r = Rouge155()

In [42]:
base = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/'
r.system_dir = base + 'system_multi'
r.model_dir = base + 'model_multi'
r.system_filename_pattern = 'SetSummary.(\d+).txt'
r.model_filename_pattern = 'SetSummary.#ID#.[A-Z].txt'

In [43]:
output = r.convert_and_evaluate()

2015-12-06 14:38:22,924 [MainThread  ] [INFO ]  Writing summaries.
INFO:global:Writing summaries.
2015-12-06 14:38:22,934 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpE7BOIZ/system and model files to /tmp/tmpE7BOIZ/model.
INFO:global:Processing summaries. Saving system files to /tmp/tmpE7BOIZ/system and model files to /tmp/tmpE7BOIZ/model.
2015-12-06 14:38:22,937 [MainThread  ] [INFO ]  Processing files in /home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/system_multi.
INFO:global:Processing files in /home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/system_multi.
2015-12-06 14:38:22,943 [MainThread  ] [INFO ]  Processing SetSummary.30024.txt.
INFO:global:Processing SetSummary.30024.txt.
2015-12-06 14:38:22,947 [MainThread  ] [INFO ]  Processing SetSummary.30034.txt.
INFO:global:Processing SetSummary.30034.txt.
2015-12-06 14:38:22,949 [MainThread  ] [INFO ]  Pr

In [44]:
print output

---------------------------------------------
1 ROUGE-1 Average_R: 0.69559 (95%-conf.int. 0.67698 - 0.71529)
1 ROUGE-1 Average_P: 0.73551 (95%-conf.int. 0.72662 - 0.74471)
1 ROUGE-1 Average_F: 0.71408 (95%-conf.int. 0.70158 - 0.72631)
---------------------------------------------
1 ROUGE-2 Average_R: 0.54948 (95%-conf.int. 0.53331 - 0.56623)
1 ROUGE-2 Average_P: 0.58096 (95%-conf.int. 0.57390 - 0.58835)
1 ROUGE-2 Average_F: 0.56405 (95%-conf.int. 0.55357 - 0.57391)
---------------------------------------------
1 ROUGE-3 Average_R: 0.50011 (95%-conf.int. 0.48520 - 0.51481)
1 ROUGE-3 Average_P: 0.52895 (95%-conf.int. 0.52158 - 0.53618)
1 ROUGE-3 Average_F: 0.51345 (95%-conf.int. 0.50283 - 0.52303)
---------------------------------------------
1 ROUGE-4 Average_R: 0.46876 (95%-conf.int. 0.45460 - 0.48249)
1 ROUGE-4 Average_P: 0.49593 (95%-conf.int. 0.48933 - 0.50245)
1 ROUGE-4 Average_F: 0.48132 (95%-conf.int. 0.47138 - 0.49027)
---------------------------------------------
1 ROUGE-L Aver

In [45]:
output_dict = r.output_to_dict(output)

In [46]:
print output_dict

{u'rouge_3_f_score_cb': 0.50283, u'rouge_3_f_score_ce': 0.52303, u'rouge_1_precision': 0.73551, u'rouge_l_precision_cb': 0.70772, u'rouge_l_f_score_cb': 0.68323, u'rouge_l_precision_ce': 0.72701, u'rouge_3_recall': 0.50011, u'rouge_3_precision_ce': 0.53618, u'rouge_w_1.2_f_score_ce': 0.31515, u'rouge_s*_f_score_ce': 0.53315, u'rouge_2_precision_ce': 0.58835, u'rouge_2_precision_cb': 0.5739, u'rouge_2_recall': 0.54948, u'rouge_w_1.2_f_score_cb': 0.30491, u'rouge_3_precision_cb': 0.52158, u'rouge_4_f_score_ce': 0.49027, u'rouge_2_precision': 0.58096, u'rouge_1_recall_cb': 0.67698, u'rouge_1_recall_ce': 0.71529, u'rouge_s*_precision': 0.55244, u'rouge_4_f_score_cb': 0.47138, u'rouge_s*_recall_cb': 0.46544, u'rouge_su*_recall_ce': 0.51966, u'rouge_w_1.2_f_score': 0.31, u'rouge_w_1.2_recall': 0.23259, u'rouge_su*_recall_cb': 0.46747, u'rouge_l_precision': 0.71719, u'rouge_2_recall_cb': 0.53331, u'rouge_s*_precision_ce': 0.56351, u'rouge_2_recall_ce': 0.56623, u'rouge_4_precision_cb': 0.4893

In [7]:
# Let's try to rename the model summaries so that they match what we know
import os
base = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/'

In [14]:
# Only run once, won't work again
modelBase = base + 'model'
for name in os.listdir(modelBase):
    tmp = name.split('.')
    newName = 'Summary.{}.{}'.format(tmp[0][1:], tmp[-1])
    newFile = os.path.join(modelBase, newName)
    print name
    oldFile = os.path.join(modelBase, name)
    os.rename(oldFile, newFile)

D30040.M.100.T.G
D30053.M.100.T.F
D30053.M.100.T.B
D30046.M.100.T.B
D31050.M.100.T.F
D31001.M.100.T.G
D30001.M.100.T.C
D30049.M.100.T.E
D30034.M.100.T.A
D30038.M.100.T.H
D30055.M.100.T.F
D30029.M.100.T.F
D30031.M.100.T.F
D30033.M.100.T.D
D30048.M.100.T.G
D30024.M.100.T.A
D30003.M.100.T.B
D30011.M.100.T.F
D30037.M.100.T.D
D30049.M.100.T.D
D31008.M.100.T.D
D31043.M.100.T.H
D30029.M.100.T.A
D30010.M.100.T.B
D30046.M.100.T.H
D30047.M.100.T.H
D31031.M.100.T.F
D30001.M.100.T.D
D30007.M.100.T.D
D30059.M.100.T.H
D30036.M.100.T.D
D30029.M.100.T.C
D31001.M.100.T.E
D31031.M.100.T.G
D31038.M.100.T.D
D30048.M.100.T.B
D31026.M.100.T.C
D30038.M.100.T.E
D31043.M.100.T.G
D31032.M.100.T.D
D30008.M.100.T.G
D30051.M.100.T.F
D31038.M.100.T.G
D30050.M.100.T.B
D30036.M.100.T.A
D30040.M.100.T.H
D30003.M.100.T.F
D31009.M.100.T.C
D30028.M.100.T.F
D30015.M.100.T.H
D30006.M.100.T.C
D30047.M.100.T.B
D30056.M.100.T.E
D30045.M.100.T.E
D30024.M.100.T.G
D30042.M.100.T.D
D30022.M.100.T.D
D31031.M.100.T.C
D31013.M.100.T

In [16]:
for name in os.listdir(modelBase):
    newName = 'Set' + name + '.txt'
    newFile = os.path.join(modelBase, newName)
    oldFile = os.path.join(modelBase, name)
    os.rename(oldFile, newFile)

In [17]:
from collections import Counter

In [28]:
# Only run once, won't work again
modelBase = base + 'model_single'
setCount = Counter() # Keeps count of documents in a set
docIDCount = Counter()
for name in os.listdir(modelBase):
    tmp = name.split('.')
    ID = tmp[0][1:]
    Person = tmp[4]
    DOCID = tmp[-2][3:] + tmp[-1]
    newName = 'Summary.{}{}.{}.txt'.format(ID,DOCID , Person)
    setCount[ID] += 1
    docIDCount[DOCID] += 1
    newFile = os.path.join(modelBase, newName)
    print name
    oldFile = os.path.join(modelBase, name)
    os.rename(oldFile, newFile)

D31032.P.10.T.D.APW19981007.0563
D30050.P.10.T.B.NYT19981002.0366
D31050.P.10.T.F.APW19981220.0356
D30011.P.10.T.A.APW19981008.0527
D30005.P.10.T.C.APW19981003.0517
D30044.P.10.T.B.APW19981117.0530
D30034.P.10.T.A.APW19981130.0497
D30022.P.10.T.A.APW19981015.0177
D30015.P.10.T.H.NYT19981004.0102
D30042.P.10.T.C.APW19981221.1004
D30020.P.10.T.C.APW19981128.0168
D30051.P.10.T.B.APW19981116.0231
D31033.P.10.T.D.NYT19981018.0089
D31032.P.10.T.F.APW19981001.0299
D31043.P.10.T.D.APW19981129.0896
D30002.P.10.T.B.APW19981031.0720
D30002.P.10.T.C.APW19981031.0720
D30042.P.10.T.F.APW19981206.0371
D30008.P.10.T.A.APW19981109.0274
D30029.P.10.T.A.APW19981228.0467
D30034.P.10.T.F.APW19981211.0990
D30047.P.10.T.H.APW19981207.0578
D30037.P.10.T.G.APW19981010.0374
D30008.P.10.T.B.APW19981019.0104
D30044.P.10.T.G.APW19981114.0575
D30034.P.10.T.F.APW19981211.0972
D30001.P.10.T.C.APW19981031.0167
D30047.P.10.T.B.APW19981120.0290
D31008.P.10.T.H.NYT19981012.0357
D30059.P.10.T.G.NYT19981122.0110
D30003.P.1

In [34]:
# Take the original documents and rename their containing forlder
# Only run once, won't work again
modelBase = base + 'docs'
_, dirs, _ = os.walk(modelBase).next()
for dir in dirs:
    ID = dir[1:-1]
    newFile = os.path.join(modelBase, ID)
    print ID
    oldFile = os.path.join(modelBase, dir)
    os.rename(oldFile, newFile)

30055
30022
30010
30034
31022
30038
30036
31001
30033
30008
31050
30046
30051
31008
30053
30001
31032
30040
31043
30042
31038
31031
30017
30027
30029
30050
30044
30007
30002
30037
30031
30049
30059
31033
30026
30011
30028
31009
30024
30006
31026
30005
30020
30003
31013
30056
30047
30045
30048
30015


In [40]:
modelBase = base + 'docs'
_, dirs, _ = os.walk(modelBase).next()
for dir in dirs:
    for name in os.listdir(os.path.join(modelBase, dir)):
        tmp = name.split('.')
        ID = tmp[0][3:] + tmp[1] + '.txt'
        newFile = os.path.join(modelBase, dir, ID)
        print newFile
        oldFile = os.path.join(modelBase, dir, name)
        os.rename(oldFile, newFile)

/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199811200887.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199810260361.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199811040772.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199811110288.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199811100432.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199810230569.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199810250187.txt
/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/cs182_data/rouge_data/docs/31038/199811120